## SRJ

## Adapt - GCIM

## First step - To create Generating functions

## Generating fns code ready.  single & 2 singles

## Gradients added and state is ready

In [1]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
theta = np.pi/2
print('Theta is', theta)

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 2.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886 
r_bohr = bond_length * 1.0  # Convert bond length to Bohr radius 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[240] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
  


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

    
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('The operator pool length is', len(operator_pool))
    states = [hf_state]


    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        counts1 = measure(k)  #Measure the current state
        print('The measurement of new state is', counts1)
        for i in operator_pool:
            print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                #print('Print, if this is activated - HF state')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)   
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))  # Get the Hamiltonian matrix
        print('Shape of Hamiltonian matrix is', Ham_matrix.shape)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                
                print('The value of i and j is', i, j)
                left_op = states[i].T.conj()
                right_op = states[j]
                M[i,j] = left_op.dot(Ham_matrix.dot(right_op)).real
                print(f'The value of i and j', i, j, 'in M', M[i,j])
                S[i,j] = left_op.dot(right_op).real
                print(f'The value of i and j', i, j, 'is', S[i,j])
                #S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                #print(f'The value of i and j', i, j, 'is', S[i,j])
                #M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy) 
        
    return ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state


Theta is 1.5707963267948966


In [2]:
ash_excitation, states,eig,gs_energy,Ham_matrix, hf_state = adaptvqe(adapt_it=2, e_th=1e-12)

HF state is -2.075242826741247
The operator pool length is 26
The adapt iteration now is 0
The measurement of new state is {'11110000': 1000}
The current excitation operator is a⁺(0) a(4)
The expectation value of a⁺(0) a(4) is 8.122859178061324e-07
The current excitation operator is a⁺(0) a(6)
The expectation value of a⁺(0) a(6) is 3.3817753229171372e-49
The current excitation operator is a⁺(1) a(5)
The expectation value of a⁺(1) a(5) is 8.122859177506212e-07
The current excitation operator is a⁺(1) a(7)


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(1) a(7) is 0.0
The current excitation operator is a⁺(2) a(4)
The expectation value of a⁺(2) a(4) is 4.206725225788884e-49
The current excitation operator is a⁺(2) a(6)
The expectation value of a⁺(2) a(6) is 6.281947146219434e-07
The current excitation operator is a⁺(3) a(5)
The expectation value of a⁺(3) a(5) is 0.0
The current excitation operator is a⁺(3) a(7)


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(3) a(7) is 6.281947146358212e-07
The current excitation operator is a⁺(0) a⁺(1) a(4) a(5)


...

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 0.0
The current excitation operator is a⁺(2) a⁺(3) a(5) a(6)


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 0.0
The current excitation operator is a⁺(2) a⁺(3) a(6) a(7)
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.05762193361120571
The highest operator value is 0.7047721989580158 for operator a⁺(2) a⁺(3) a(4) a(5)
Highest gradient excitation is [2, 3, 4, 5]
ash_excitation is [[2, 3, 4, 5], [2, 3, 4, 5]]
The length of ash_excitation before generating matrix is 2
0: ─╭|Ψ⟩─────────────────────┤  State
1: ─├|Ψ⟩─────────────────────┤  State
2: ─├|Ψ⟩─╭G²(1.57)─╭G²(1.57)─┤  State
3: ─├|Ψ⟩─├G²(1.57)─├G²(1.57)─┤  State
4: ─├|Ψ⟩─├G²(1.57)─├G²(1.57)─┤  State
5: ─├|Ψ⟩─╰G²(1.57)─╰G²(1.57)─┤  State
6: ─├|Ψ⟩─────────────────────┤  State
7: ─╰|Ψ⟩─────────────────────┤  State
State count after Generating function is {'11001100': 1000}
After adding new state, the length of states is 3
Shape of Hamiltonian matrix is (256, 256)
The value of i and j is 0 0
The value of i and j 0 0 in M (-2.0752428267412473+0j)
The value of i and j 0 0 is (0.999999999999999

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


## Preparing the state

In [4]:
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def stateprep(i):
    qml.StatePrep(hf_state, wires=range(qubits))
    if len(ash_excitation[i]) == 4:
        print('DE in stateprep is ', ash_excitation[i])
        qml.DoubleExcitation(theta, wires = ash_excitation[i])
    elif len(ash_excitation[i]) == 2:
        qml.SingleExcitation(theta, wires = ash_excitation[i])
    print('Applied wires:', ash_excitation[i])
    return qml.state()

#Measurement of state
dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev_meas)
def measure(ostate):
    qml.StatePrep(ostate, wires=range(qubits))
    return qml.counts()


## Operator created

In [5]:
from pennylane.fermi import from_string

def operator(ash_excitation, theta, i):
    singles_fermi = []
    doubles_fermi = []
    doubles_pauli = []
    singles_pauli = []
    if len(ash_excitation[i]) == 4:
        print('DE in operator calc is ', ash_excitation[i])
        doubles_fermi.append(from_string(f"{ash_excitation[i][3]}+ {ash_excitation[i][2]}+ {ash_excitation[i][1]}- {ash_excitation[i][0]}-")
                       - from_string(f"{ash_excitation[i][0]}+ {ash_excitation[i][1]}+ {ash_excitation[i][2]}- {ash_excitation[i][3]}-"))
        print('Doubles fermi right now is', doubles_fermi[-1])
        doubles_pauli.append(qml.jordan_wigner(doubles_fermi[-1], qubits))
        k1 = qml.exp((theta/2 * doubles_pauli[0]).operation()) #Removed the range(qubits) part
    elif len(ash_excitation[i]) == 2:
        print('SE in operator calc is ', ash_excitation[i])
        singles_fermi.append(from_string(f"{ash_excitation[i][1]}+ {ash_excitation[i][0]}-")
                       - from_string(f"{ash_excitation[i][0]}+ {ash_excitation[i][1]}-"))
        print('Singles fermi right now is', singles_fermi[-1])
        singles_pauli.append(qml.jordan_wigner(singles_fermi[-1], qubits))
        k1 = qml.exp((theta/2 * singles_pauli[0]).operation()) #Removed the range(qubits) part
    exp_matrix1 = qml.matrix(k1, wire_order=range(qubits))
    print('exp matrix1', exp_matrix1.shape)
    return exp_matrix1

## Gradient calculation

In [6]:
#ash_excitation = []
Hmat = qml.matrix(H)
#np.set_printoptions(threshold=np.inf)

#dev = qml.device("lightning.qubit", wires=qubits)
#@qml.qnode(dev)
def gcm_grad(ash_excitation, H, theta):

    #
    L = len(ash_excitation)
    print('Value of L which is the excitation',L) #This is right 
    Heff = np.zeros((L,L),dtype=complex)
    ovlp = np.zeros((L,L),dtype=complex)
    d_Heff = np.zeros((L,L),dtype=complex)
    d_ovlp = np.zeros((L,L),dtype=complex)
    print('The shape of Heff is', Heff.shape)
    for i in range(L):
        for j in range(L):
            print(f'The value of i {i} and j is  {j}')
            _ = (stateprep(i).conj().T) # G(ash_excitation[i], theta)@hf_state
            counts = measure(_)
            print('States of the stateprep especially bra vector is', counts)
            #print('The bra vector is', _) 
            print('j value going in', j)
            counts1 = measure(stateprep(j))
            print('States of the stateprep especially ket vector is', counts1)
            Heff[i,j] = _.dot(Hmat.dot(stateprep(j))).real
            print(f'The value of Heff[{i},{j}] is', Heff[i,j])
            ovlp[i,j] = _.dot(stateprep(j)).real
            print(f'The value of ovlp[{i},{j}] is', ovlp[i,j])
            if i == L-1 and j != L-1:
                print('Case 1 is activated')
                print(f'The value of i {i} and j is  {j}')
                #d_Heff[i,j] = -1*_.dot((ash_excitation[i]@H).dot(stateprep[j]))[0,0].real
                d_Heff[i,j] = -1*_.dot((operator(ash_excitation, theta, i)@Hmat).dot(stateprep(j))).real
                print('The value of d_Heff in case 1 is', d_Heff[i,j])
                d_ovlp[i,j] = _.dot((operator(ash_excitation, theta, i)).dot(stateprep(j))).real
                print('The value of d_ovlp in case 1 is', d_ovlp[i,j])

            elif i != L-1 and j == L-1:
                print('Case 2 is activated')
                print(f'The value of i {i} and j is  {j}')
                d_Heff[i,j] = _.dot((Hmat@(operator(ash_excitation, theta, i))).dot(stateprep(j))).real
                print('The value of d_Heff in case 2 is', d_Heff[i,j])
                d_ovlp[i,j] = _.dot(operator(ash_excitation, theta, i).dot(stateprep(j))).real
                print('The value of d_ovlp in case 1 is', d_ovlp[i,j])

                
            elif i != L-1 and j != L-1:
                print('Case 3 is activated')
                print(f'The value of i {i} and j is  {j}')
                #d_Heff[i,j] = _.dot((Hmat@ash_excitation[i]-ash_excitation[i]@Hmat).dot(ash_excitation[j]@hf))[0,0].real
                d_Heff[i,j] = _.dot((Hmat@(operator(ash_excitation, theta, i))-(operator(ash_excitation, theta, i))@Hmat).dot(stateprep(j))).real
            print('Moving to next iteration of j')
            print('\n')
        print('Moving to next iteration of i')
        print('\n')
    #e,f = sl.eig(Heff,ovlp)
    e,f=scipy.linalg.eigh(Heff,ovlp)
    print('Eigenvalues are', e)
    idx = np.argsort(e)
    f = f[:,idx]
    H_k = np.conj(f[0]).T@Heff@f[0]
    S_k = np.conj(f[0]).T@ovlp@f[0]
    d_H_k = np.conj(f[0]).T@d_Heff@f[0]
    d_S_k = np.conj(f[0]).T@d_ovlp@f[0]
    g = (d_H_k*S_k - H_k*d_S_k)/(S_k*S_k)
    print('The value of g:',g)

gcm_grad(ash_excitation, H, theta)

Value of L which is the excitation 2
The shape of Heff is (2, 2)
The value of i 0 and j is  0
DE in stateprep is  [2, 3, 6, 7]
Applied wires: [2, 3, 6, 7]
States of the stateprep especially bra vector is {'11000011': 138, '11110000': 862}
j value going in 0
DE in stateprep is  [2, 3, 6, 7]
Applied wires: [2, 3, 6, 7]
States of the stateprep especially ket vector is {'11000011': 132, '11110000': 868}
DE in stateprep is  [2, 3, 6, 7]
Applied wires: [2, 3, 6, 7]
The value of Heff[0,0] is (-1.420376125683405+0j)
DE in stateprep is  [2, 3, 6, 7]
Applied wires: [2, 3, 6, 7]
The value of ovlp[0,0] is (0.9999999999999994+0j)
Case 3 is activated
The value of i 0 and j is  0
DE in operator calc is  [2, 3, 6, 7]
Doubles fermi right now is 1.0 * a⁺(7) a⁺(6) a(3) a(2)
+ -1.0 * a⁺(2) a⁺(3) a(6) a(7)
exp matrix1 (256, 256)
DE in operator calc is  [2, 3, 6, 7]
Doubles fermi right now is 1.0 * a⁺(7) a⁺(6) a(3) a(2)
+ -1.0 * a⁺(2) a⁺(3) a(6) a(7)
exp matrix1 (256, 256)
DE in stateprep is  [2, 3, 6, 7]
A

In [4]:
ash_excitation[1]
ash_excitation

[[2, 3, 6, 7],
 [0, 3, 5, 6],
 [0, 1, 4, 5],
 [1, 2, 4, 7],
 [0, 3, 5, 6],
 [1, 2, 4, 7],
 [0, 4]]

In [8]:
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def summa():
    qml.StatePrep(hf_state, wires=range(qubits))
    for i in range(len(ash_excitation)):
        print('The value of i is', i)
        if len(ash_excitation[i]) == 4:
            print('DE in summa is ', ash_excitation[i])
            qml.DoubleExcitation(theta, wires=ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            print('SE in summa is ', ash_excitation[i])
            qml.SingleExcitation(theta, wires=ash_excitation[i])
    return qml.expval(H)

energy = summa()
print(f'The energy of the excitation', energy)

The value of i is 0
DE in summa is  [2, 3, 6, 7]
The value of i is 1
DE in summa is  [0, 3, 5, 6]
The value of i is 2
DE in summa is  [0, 1, 4, 5]
The value of i is 3
DE in summa is  [1, 2, 4, 7]
The value of i is 4
DE in summa is  [0, 3, 5, 6]
The value of i is 5
DE in summa is  [1, 2, 4, 7]
The value of i is 6
SE in summa is  [0, 4]
The energy of the excitation -1.8004043164964592
